### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish Arabic"

# parameter
file = "Turkish Arabic SV"  # Turkish Dutch SV
sheets = "2 gram hybrid"  # 2 gram target, 2 gram hybrid
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6
sample_num = 6

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [7]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [8]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [10]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram_pair1
0,acele cevap
1,acele karar
2,acele kararlar
3,adam falan
4,adam haklı
...,...
271,zaten sabah
272,zaten tamam
273,zavallı adam
274,zavallı herif


In [11]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
        else:
            pass

In [12]:
df_search_result = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,acele cevap,839.920,848.200,ne demiş öpücükler öperim seni sevilen annen d...,uLM3hC_A9B0
1,acele karar,3660.420,3662.420,böyle acele karar verme biraz düşün,z0cVY9navGM
2,acele karar,5732.164,5733.899,acele karar verme istersen,1mQL15DyOcM
3,acele karar,5236.805,5238.925,acele karar vermek,F85pyD5KTWE
4,acele karar,5682.717,5689.920,i dare ediyorduk be kardeşim acele karar verdi...,i7bA1qFLC_E
...,...,...,...,...,...
939,zavallı adam,5638.880,5642.560,zavallı adam koca bir bebek gibi ağladı ne tat...,4OO3M5j0lbg
940,ziyaretçi kabul,50.100,51.600,ziyaretçi kabul etmeyin,UY03fnBmxK8
941,ziyaretçi kabul,2265.280,2267.280,ziyaretçi kabul etmiyoruz,204vg0ZpzxU
942,ziyaretçi kabul,8649.404,8651.028,ziyaretçi kabul etmiyor muzaffer bey,aMWKWcdyYMo


In [13]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,acele cevap,846.670000,847.840000,ne demiş öpücükler öperim seni sevilen annen d...,uLM3hC_A9B0
1,acele karar,3660.705714,3661.448571,böyle acele karar verme biraz düşün,z0cVY9navGM
2,acele karar,5732.164000,5732.964769,acele karar verme istersen,1mQL15DyOcM
3,acele karar,5236.805000,5238.218333,acele karar vermek,F85pyD5KTWE
4,acele karar,5684.637800,5685.529600,i dare ediyorduk be kardeşim acele karar verdi...,i7bA1qFLC_E
...,...,...,...,...,...
939,zavallı adam,5638.880000,5639.719298,zavallı adam koca bir bebek gibi ağladı ne tat...,4OO3M5j0lbg
940,ziyaretçi kabul,50.100000,51.143478,ziyaretçi kabul etmeyin,UY03fnBmxK8
941,ziyaretçi kabul,2265.280000,2266.560000,ziyaretçi kabul etmiyoruz,204vg0ZpzxU
942,ziyaretçi kabul,8649.404000,8650.125778,ziyaretçi kabul etmiyor muzaffer bey,aMWKWcdyYMo


In [14]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,acele cevap,846.070000,848.440000,ne demiş öpücükler öperim seni sevilen annen d...,uLM3hC_A9B0
1,acele karar,3660.105714,3662.048571,böyle acele karar verme biraz düşün,z0cVY9navGM
2,acele karar,5731.564000,5733.564769,acele karar verme istersen,1mQL15DyOcM
3,acele karar,5236.205000,5238.818333,acele karar vermek,F85pyD5KTWE
4,acele karar,5684.037800,5686.129600,i dare ediyorduk be kardeşim acele karar verdi...,i7bA1qFLC_E
...,...,...,...,...,...
939,zavallı adam,5638.280000,5640.319298,zavallı adam koca bir bebek gibi ağladı ne tat...,4OO3M5j0lbg
940,ziyaretçi kabul,49.500000,51.743478,ziyaretçi kabul etmeyin,UY03fnBmxK8
941,ziyaretçi kabul,2264.680000,2267.160000,ziyaretçi kabul etmiyoruz,204vg0ZpzxU
942,ziyaretçi kabul,8648.804000,8650.725778,ziyaretçi kabul etmiyor muzaffer bey,aMWKWcdyYMo


In [15]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,acele cevap,846,848,ne demiş öpücükler öperim seni sevilen annen d...,uLM3hC_A9B0
1,acele karar,3660,3662,böyle acele karar verme biraz düşün,z0cVY9navGM
2,acele karar,5732,5734,acele karar verme istersen,1mQL15DyOcM
3,acele karar,5236,5239,acele karar vermek,F85pyD5KTWE
4,acele karar,5684,5686,i dare ediyorduk be kardeşim acele karar verdi...,i7bA1qFLC_E
...,...,...,...,...,...
939,zavallı adam,5638,5640,zavallı adam koca bir bebek gibi ağladı ne tat...,4OO3M5j0lbg
940,ziyaretçi kabul,50,52,ziyaretçi kabul etmeyin,UY03fnBmxK8
941,ziyaretçi kabul,2265,2267,ziyaretçi kabul etmiyoruz,204vg0ZpzxU
942,ziyaretçi kabul,8649,8651,ziyaretçi kabul etmiyor muzaffer bey,aMWKWcdyYMo


In [16]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,acele cevap,846,848,ne demiş öpücükler öperim seni sevilen annen d...,uLM3hC_A9B0,https://www.youtube.com/watch?v=uLM3hC_A9B0&t=...
1,acele karar,3660,3662,böyle acele karar verme biraz düşün,z0cVY9navGM,https://www.youtube.com/watch?v=z0cVY9navGM&t=...
2,acele karar,5732,5734,acele karar verme istersen,1mQL15DyOcM,https://www.youtube.com/watch?v=1mQL15DyOcM&t=...
3,acele karar,5236,5239,acele karar vermek,F85pyD5KTWE,https://www.youtube.com/watch?v=F85pyD5KTWE&t=...
4,acele karar,5684,5686,i dare ediyorduk be kardeşim acele karar verdi...,i7bA1qFLC_E,https://www.youtube.com/watch?v=i7bA1qFLC_E&t=...
...,...,...,...,...,...,...
939,zavallı adam,5638,5640,zavallı adam koca bir bebek gibi ağladı ne tat...,4OO3M5j0lbg,https://www.youtube.com/watch?v=4OO3M5j0lbg&t=...
940,ziyaretçi kabul,50,52,ziyaretçi kabul etmeyin,UY03fnBmxK8,https://www.youtube.com/watch?v=UY03fnBmxK8&t=50s
941,ziyaretçi kabul,2265,2267,ziyaretçi kabul etmiyoruz,204vg0ZpzxU,https://www.youtube.com/watch?v=204vg0ZpzxU&t=...
942,ziyaretçi kabul,8649,8651,ziyaretçi kabul etmiyor muzaffer bey,aMWKWcdyYMo,https://www.youtube.com/watch?v=aMWKWcdyYMo&t=...


In [17]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_{sheets.capitalize()}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

#### Copy Move And Delete

In [18]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx")
output_file

['Turkish_Arabic_2 gram hybrid_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [19]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [20]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass